In [1]:
import pandas as pd
import numpy as np

In [57]:
!pip install -U pandas-profiling
!pip install --upgrade pythainlp
!pip install pyLDAvis
!pip install scipy
!pip install pyLDAvis==2.1.2

Requirement already up-to-date: pandas-profiling in c:\users\chayanun c\anaconda3\lib\site-packages (2.12.0)
Requirement already up-to-date: pythainlp in c:\users\chayanun c\anaconda3\lib\site-packages (2.3.1)


In [58]:
import pythainlp
import pyLDAvis
import gensim
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
#import warnings
#warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [59]:
import pandas as pd

In [60]:
df = pd.read_csv('CustomerReviews.csv')

In [61]:
df.head()

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating
0,1,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,7b16469831074f7abc7824745ee75212,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,5.0
1,2,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,pakkaramonpondej,รสชาติเหมือนทุกสาขา แต่สาขานี้ บริการดี ที่นั่งดี,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,5.0
2,3,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,saanowy,ชาบูพรีเมี่ยมสไตล์ญี่ปุ่น เนื้อดีมากกก,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,4.0
3,4,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ployynp,เนื้อดี ไอติมดี คุ้มค่าดี,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,4.0
4,5,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,665a902b335b434ab489f00f2fbb477e,อาหารมีคุณภาพ บริการดีค่ะ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,5.0


In [62]:
def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    merged = merged + ',' + word
  return merged[1:]

In [63]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [64]:
df.tail()

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating,Review_tokenized
16,17,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,ployynp,บุฟเฟ่ต์ชาบูและพิซซ่าไม่อั้นในราคา 199 บาท เน้...,หลังจากที่เคยลองสาขายูเนี่ยนมอลล์ไป รอบนี้มาที...,4.0,"หลังจากที่,เคย,ลอง,สาขา,ยู,เนี่ย,นม,อลล์,ไป, ,..."
17,18,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,27a91236fe5e4559a4f097c97a480781,ร้านบุฟเฟ่ต์ ราคามิตรภาพ อยู่ชั้น4 ติดโรงหนัง ...,ร้านบุฟเฟ่ต์ที่มีโปรโมชั่นหัวละ199บาท ไม่รวมน้...,4.0,"ร้าน,บุฟเฟ่ต์,ที่,มี,โปรโมชั่น,หัว,ละ,199,บาท,..."
18,19,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,0b81d251e4db486f9bcdba73b374ed99,ของหลากหลาย ปนๆ งงๆ นิดหน่อย,เคยรู้จักร้านนี้จากที่ union mall ไม่เคยได้ลอง...,3.0,"เคย,รู้จัก,ร้าน,นี้,จาก,ที่, ,union, ,mall, ,ไ..."
19,20,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,40e0e087f3914fd49a8933b5a29936ca,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,5.0,"อร่อย,มาก,ค่ะ, ,คุ้มค่า,สมราคา, ,บุ,ฟเฟ่,หมู,ผ..."
20,21,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,41841cb99ea243a3a8d4b006e946c586,แม้จะแปลกบ้าง แต่ก็ถือว่าอยู่ในเกณฑ์ที่ดี มีอา...,ก็ตามที่เขียนเลยครับ ว่า ถ้าจะจ่ายในราคา 199 บ...,NaN,"ก็,ตามที่,เขียน,เลย,ครับ, ,ว่า, ,ถ้า,จะ,จ่าย,ใ..."


In [65]:
documents = df['Review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [45]:
print(dictionary.token2id.keys())

dict_keys(['\n', ' ', '1', '4', '560', 'กัน', 'กับ', 'การ', 'กิน', 'ก็', 'ก่อน', 'ของ', 'ข้างนอก', 'ความต้องการ', 'คอ', 'คือ', 'คุณ', 'คุณภาพ', 'คุ้ม', 'ค่อนข้าง', 'จำกัด', 'ซุป', 'ดำ', 'ดี', 'ดีมาก', 'ถาด', 'ถาม', 'ถือว่า', 'ที่', 'นะ', 'นิดๆ', 'นี้', 'น่ารัก', 'น้ำ', 'น้ำจิ้ม', 'บริการ', 'บาท', 'ประทับใจ', 'พนักงาน', 'พอกับ', 'ภาพ', 'มา', 'มาก', 'รวม', 'รสชาติ', 'ราคา', 'ร้าน', 'ละ', 'ว่า', 'สรุป', 'สะอาดสะอ้าน', 'สัน', 'สำคัญ', 'สุด', 'หลากหลาย', 'อร่อย', 'ออก', 'อันนี้', 'อาหาร', 'เคย', 'เคลม', 'เซ็ท', 'เต้าเจี้ยว', 'เนื้อ', 'เป็ด', 'เป็น', 'เผ็ด', 'เพราะว่า', 'เรา', 'แต่', 'แต่เพียง', 'แบบนี้', 'แปลก', 'แฟน', 'และ', 'โดยรวม', 'โมจิ', 'ใน', 'ใส่ใจ', 'ใหม่', 'ได้', 'ไป', 'ไม่', 'ไม่ต้อง', 'ไอติม', 'ๆ', '\u200b', '           ', '21', '30', '9', 'CTW', 'Terminal', 'กา', 'การงด', 'ก็ได้', 'ขอ', 'คิดถึง', 'จริงๆ', 'จะ', 'ชาบู', 'ญี่ปุ่น', 'ฐาน', 'ต้อง', 'ทะยอย', 'ทาง', 'ทำงาน', 'ที่ทาง', 'นาที', 'นึกถึง', 'ปิด', 'พนักงานบริการ', 'พระราม', 'พรีเมียม', 'พร้อม', 'พาราไดซ์', 'มาตรา', 'มี', 

In [46]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]

In [47]:
print(gensim_corpus)

[[(0, 6), (1, 5), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 5), (9, 4), (10, 1), (11, 2), (12, 1), (13, 1), (14, 1), (15, 2), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 2), (24, 1), (25, 1), (26, 1), (27, 1), (28, 2), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 3), (36, 1), (37, 2), (38, 1), (39, 1), (40, 1), (41, 1), (42, 2), (43, 1), (44, 3), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 2), (53, 1), (54, 1), (55, 3), (56, 1), (57, 1), (58, 3), (59, 1), (60, 1), (61, 1), (62, 1), (63, 2), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 2), (81, 1), (82, 4), (83, 1), (84, 1), (85, 2), (86, 12)], [(0, 1), (1, 27), (7, 1), (9, 5), (10, 1), (15, 1), (17, 1), (23, 2), (31, 2), (32, 1), (35, 2), (38, 1), (42, 1), (46, 4), (58, 1), (59, 1), (65, 1), (68, 1), (69, 2), (80, 1), (82, 4), (87, 1), (88, 1), (89, 1), (90, 1), 

In [48]:
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

In [49]:
print(word_frequencies)

[[('\n', 6), (' ', 5), ('1', 1), ('4', 1), ('560', 1), ('กัน', 1), ('กับ', 2), ('การ', 1), ('กิน', 5), ('ก็', 4), ('ก่อน', 1), ('ของ', 2), ('ข้างนอก', 1), ('ความต้องการ', 1), ('คอ', 1), ('คือ', 2), ('คุณ', 1), ('คุณภาพ', 1), ('คุ้ม', 1), ('ค่อนข้าง', 1), ('จำกัด', 1), ('ซุป', 1), ('ดำ', 1), ('ดี', 2), ('ดีมาก', 1), ('ถาด', 1), ('ถาม', 1), ('ถือว่า', 1), ('ที่', 2), ('นะ', 1), ('นิดๆ', 1), ('นี้', 1), ('น่ารัก', 1), ('น้ำ', 1), ('น้ำจิ้ม', 1), ('บริการ', 3), ('บาท', 1), ('ประทับใจ', 2), ('พนักงาน', 1), ('พอกับ', 1), ('ภาพ', 1), ('มา', 1), ('มาก', 2), ('รวม', 1), ('รสชาติ', 3), ('ราคา', 1), ('ร้าน', 1), ('ละ', 1), ('ว่า', 1), ('สรุป', 1), ('สะอาดสะอ้าน', 1), ('สัน', 1), ('สำคัญ', 2), ('สุด', 1), ('หลากหลาย', 1), ('อร่อย', 3), ('ออก', 1), ('อันนี้', 1), ('อาหาร', 3), ('เคย', 1), ('เคลม', 1), ('เซ็ท', 1), ('เต้าเจี้ยว', 1), ('เนื้อ', 2), ('เป็ด', 1), ('เป็น', 1), ('เผ็ด', 1), ('เพราะว่า', 1), ('เรา', 1), ('แต่', 1), ('แต่เพียง', 1), ('แบบนี้', 1), ('แปลก', 1), ('แฟน', 1), ('และ', 1), ('โดย

In [50]:
num_topics = 20
chunksize = 1000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

Wall time: 516 ms


In [69]:
print(id2word)

{0: '\n', 1: ' ', 2: '1', 3: '4', 4: '560', 5: 'กัน', 6: 'กับ', 7: 'การ', 8: 'กิน', 9: 'ก็', 10: 'ก่อน', 11: 'ของ', 12: 'ข้างนอก', 13: 'ความต้องการ', 14: 'คอ', 15: 'คือ', 16: 'คุณ', 17: 'คุณภาพ', 18: 'คุ้ม', 19: 'ค่อนข้าง', 20: 'จำกัด', 21: 'ซุป', 22: 'ดำ', 23: 'ดี', 24: 'ดีมาก', 25: 'ถาด', 26: 'ถาม', 27: 'ถือว่า', 28: 'ที่', 29: 'นะ', 30: 'นิดๆ', 31: 'นี้', 32: 'น่ารัก', 33: 'น้ำ', 34: 'น้ำจิ้ม', 35: 'บริการ', 36: 'บาท', 37: 'ประทับใจ', 38: 'พนักงาน', 39: 'พอกับ', 40: 'ภาพ', 41: 'มา', 42: 'มาก', 43: 'รวม', 44: 'รสชาติ', 45: 'ราคา', 46: 'ร้าน', 47: 'ละ', 48: 'ว่า', 49: 'สรุป', 50: 'สะอาดสะอ้าน', 51: 'สัน', 52: 'สำคัญ', 53: 'สุด', 54: 'หลากหลาย', 55: 'อร่อย', 56: 'ออก', 57: 'อันนี้', 58: 'อาหาร', 59: 'เคย', 60: 'เคลม', 61: 'เซ็ท', 62: 'เต้าเจี้ยว', 63: 'เนื้อ', 64: 'เป็ด', 65: 'เป็น', 66: 'เผ็ด', 67: 'เพราะว่า', 68: 'เรา', 69: 'แต่', 70: 'แต่เพียง', 71: 'แบบนี้', 72: 'แปลก', 73: 'แฟน', 74: 'และ', 75: 'โดยรวม', 76: 'โมจิ', 77: 'ใน', 78: 'ใส่ใจ', 79: 'ใหม่', 80: 'ได้', 81: 'ไป', 82: 'ไม่'

LDAvis
LDAvis is a web-based interactive visualisation of topics estimated using LDA. It provides a global view of the topics (and how they differ from each other), while at the same time allowing for a deep inspection of the terms most highly associated with each individual topic.

The package extracts information from a fitted LDA topic model to inform an interactive web-based visualization.

The visualisation has two basic pieces.

The left panel visualise the topics as circles in the two-dimensional plane whose centres are determined by computing the Jensen–Shannon divergence between topics, and then by using multidimensional scaling to project the inter-topic distances onto two dimensions. Each topic’s overall prevalence is encoded using the areas of the circles.

The right panel depicts a horizontal bar chart whose bars represent the individual terms that are the most useful for interpreting the currently selected topic on the left. A pair of overlaid bars represent both the corpus-wide frequency of a given term as well as the topic-specific frequency of the term.

The λ slider allows to rank the terms according to term relevance. By default, the terms of a topic are ranked in decreasing order according their topic-specific probability ( λ = 1 ). Moving the slider allows to adjust the rank of terms based on much discriminatory (or "relevant") are for the specific topic. The suggested “optimal” value of λ is 0.6.

In [84]:
import pyLDAvis.gensim as gensim
gensim.prepare(model, gensim_corpus, dictionary, mds='tsne')

C:\Users\Chayanun C\anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
C:\Users\Chayanun C\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:348: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  error = np.finfo(np.float).max
C:\Users\Chayanun C\anaconda3\lib\site-p

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
1      -10.878068   10.538728       1        1  25.669578
15     -60.884026  -75.591820       2        1  25.401857
2       -2.336119  -53.289074       3        1  24.679943
6      -61.385319  -12.914634       4        1   7.682887
18     -28.737743  126.296997       5        1   4.235549
4        1.066606 -123.552811       6        1   3.392938
12      45.140720   -7.947869       7        1   2.989813
19    -140.795975  -98.530548       8        1   2.454811
14      61.531681  -83.240562       9        1   1.701274
9      -72.563057 -140.396881      10        1   1.694124
0       15.984491   63.578186      11        1   0.010693
16    -160.851624   68.945145      12        1   0.009761
7       85.961830   51.922935      13        1   0.009752
5      -99.020836  109.936348      14        1   0.009748
17    -176.764130   -8.681684      15        1   0.009729
11    -117.189880  -37.420200      16        1   0.009543
8      108.829918  -22.946203      17        1   0.009500
13      47.588097  121.934814      18        1   0.009500
3     -106.467834   30.176889      19        1   0.009500
10     -50.613804   59.328842      20        1   0.009500, topic_info=        Term       Freq       Total Category  logprob  loglift
1             497.00000  497.000000  Default  30.0000  30.0000
0         \n   95.00000   95.000000  Default  29.0000  29.0000
9         ก็   40.00000   40.000000  Default  28.0000  28.0000
85         ๆ   33.00000   33.000000  Default  27.0000  27.0000
162      ค่ะ   27.00000   27.000000  Default  26.0000  26.0000
..       ...        ...         ...      ...      ...      ...
541  ห่างหาย    0.00037    1.339797  Topic20  -6.7154   1.0661
567     แป้ง    0.00037    1.339792  Topic20  -6.7154   1.0661
528  วันหยุด    0.00037    1.339787  Topic20  -6.7154   1.0661
518     มื้อ    0.00037    1.339797  Topic20  -6.7154   1.0661
519      ยัง    0.00037    2.288124  Topic20  -6.7154   0.5309

[1172 rows x 6 columns], token_table=      Topic      Freq Term
term                      
146       3  1.012672     
0         1  0.273615   \n
0         2  0.326233   \n
0         3  0.263091   \n
0         5  0.052618   \n
...     ...       ...  ...
85        5  0.059336    ๆ
85        7  0.029668    ๆ
86        5  0.938111    ​
790       9  1.050509    😆
791       9  1.050555    🤣

[869 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 16, 3, 7, 19, 5, 13, 20, 15, 10, 1, 17, 8, 6, 18, 12, 9, 14, 4, 11])

import pyLDAvis
import pyLDAvis.gensim
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

%matplotlib inline
data = gensimvis.prepare(model, gensim_corpus, dictionary)
pyLDAvis.enable_notebook()
pyLDAvis.display(data)

In [93]:
model.show_topic(18)

[('\u200b', 0.067974396),
 ('\n', 0.0341331),
 (' ', 0.028499935),
 ('กิน', 0.02848694),
 ('ก็', 0.022846743),
 ('ไม่', 0.022846332),
 ('บริการ', 0.017205302),
 ('อร่อย', 0.017205084),
 ('อาหาร', 0.017205017),
 ('รสชาติ', 0.017204357)]

In [89]:
df['topics'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [92]:
df[['Restaurant', 'Review', 'topics', 'score']]

,Restaurant,Review,topics,score
0,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,18,0.996631
1,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,15,0.996441
2,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,2,0.998786
3,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,12,0.995240
4,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,1,0.996221
5,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ร้านบุฟเฟ่ ชาบูแนวญี่ปุ่น สายเนื้อหมู เนื้อวัว...,1,0.994092
6,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,Number 20 : โมโม – พาราไดส์ (สาขาเดอะมอลบางกะป...,15,0.996524
7,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ร้านชาบูเฟรนไชส์รสชาติดีมากคุ้มค่าเหมาะสมกับรา...,2,0.466849
8,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,มา านที่ขาบูชิต้องมาตอนหิว ไม่งั้นจะไม่คุ้มนะค...,1,0.994788
9,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,ใครชอบกุ้งทอดเทมปุระ แค่กุ้งเทมปุระก็คุ้มแล้ว ...,4,0.995633
